In [ ]:
!apt-get install python3.9

!sudo rm /usr/local/bin/python
!sudo ln -s /usr/bin/python3.9 /usr/local/bin/python

!sudo update-alternatives --install /usr/bin/python python /usr/bin/python3.9 1


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.9 is already the newest version (3.9.19-1+jammy1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install torch
!pip install gluonnlp pandas tqdm
!pip3 install mxnet-mkl==1.6.0 numpy==1.23.1
!pip install mxnet

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-37qjb2yj/kobert-tokenizer_67acf885f80848da8f4c5e72f7849003
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-37qjb2yj/kobert-tokenizer_67acf885f80848da8f4c5e72f7849003
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [ ]:
from transformers import AutoTokenizer, AutoModel, pipeline

BASEMODEL_NAME = "skt/kobert-base-v1"
tokenizer = AutoTokenizer.from_pretrained(BASEMODEL_NAME)
model = AutoModel.from_pretrained(BASEMODEL_NAME)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from tqdm.auto import tqdm as tqdm_auto
from tqdm import tqdm
import os
import gluonnlp as nlp

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
df = pd.read_excel("/content/drive/MyDrive/dev/Telloment/sentiment-classification/한국어_단발성_대화_데이터셋.xlsx")
print(f'column: {df.columns}')
df = df.drop(df.columns[2:], axis=1)
tt = 'text'
lb = 'labels'
df.rename(columns = {'Sentence' : tt}, inplace = True)
df.rename(columns = {'Emotion' : lb}, inplace = True)
label = df[lb].unique()
df.loc[(df['labels'] == "공포"), 'labels'] = 0
df.loc[(df['labels'] == "분노"), 'labels'] = 0
df.loc[(df['labels'] == "혐오"), 'labels'] = 0
df.loc[(df['labels'] == "슬픔"), 'labels'] = 1
df.loc[(df['labels'] == "놀람"), 'labels'] = 3
df.loc[(df['labels'] == "중립"), 'labels'] = 3
df.loc[(df['labels'] == "행복"), 'labels'] = 2


column: Index(['Sentence', 'Emotion', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', '공포',
       5468],
      dtype='object')


In [ ]:
df2_train = pd.read_excel("/content/drive/MyDrive/dev/Telloment/sentiment-classification/감성대화말뭉치(최종데이터)_Training.xlsx")
df2_test = pd.read_excel("/content/drive/MyDrive/dev/Telloment/sentiment-classification/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [ ]:
print(df2_train.columns)
print(df2_test['감정_대분류'].unique())
print(df2_train['감정_대분류'].unique())
df2_train.loc[(df2_train['감정_대분류'] == "불안"), '감정_대분류'] = 0
df2_train.loc[(df2_train['감정_대분류'] == "슬픔"), '감정_대분류'] = 1
df2_train.loc[(df2_train['감정_대분류'] == "당황"), '감정_대분류'] = 1
df2_train.loc[(df2_train['감정_대분류'] == "기쁨"), '감정_대분류'] = 2
df2_train.loc[(df2_train['감정_대분류'] == "분노"), '감정_대분류'] = 0
df2_train.loc[(df2_train['감정_대분류'] == "상처"), '감정_대분류'] = 1
# df2_train to number
df2_test.loc[(df2_test['감정_대분류'] == "불안"), '감정_대분류'] = 0
df2_test.loc[(df2_test['감정_대분류'] == "슬픔"), '감정_대분류'] = 1
df2_test.loc[(df2_test['감정_대분류'] == "당황"), '감정_대분류'] = 1
df2_test.loc[(df2_test['감정_대분류'] == "기쁨"), '감정_대분류'] = 2
df2_test.loc[(df2_test['감정_대분류'] == "분노"), '감정_대분류'] = 0
df2_test.loc[(df2_test['감정_대분류'] == "상처"), '감정_대분류'] = 1

Index(['Unnamed: 0', '연령', '성별', '상황키워드', '신체질환', '감정_대분류', '감정_소분류', '사람문장1',
       '시스템문장1', '사람문장2', '시스템문장2', '사람문장3', '시스템문장3'],
      dtype='object')
['불안' '슬픔' '당황' '기쁨' '분노' '상처']
['분노' '기쁨' '불안' '당황' '슬픔' '상처']


In [ ]:
data_list2_train = []
for q, label in zip(df2_train['사람문장1'], df2_train['감정_대분류']):
    data = []
    data.append(q)
    data.append(label)
    data_list2_train.append(data)

data_list2_valid = []
for q, label in zip(df2_test['사람문장1'], df2_test['감정_대분류']):
    data = []
    data.append(q)
    data.append(label)
    data_list2_valid.append(data)

In [ ]:
data_list = []
for q, label in zip(df[tt], df[lb]):
    data = []
    data.append(q)
    data.append(label)
    data_list.append(data)

In [ ]:
data_list.extend(data_list2_train)
data_list.extend(data_list2_valid)
print(len(data_list))

96865


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):

        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5
output_dir = '/content/drive/MyDrive/dev/Telloment/sentiment-classification/result_dual_dataset/'

In [ ]:
from transformers import AutoModel, AutoTokenizer

kobert = AutoModel.from_pretrained("skt/kobert-base-v1")

result = tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
kobert_vocab = tokenizer.get_vocab()
print(kobert_vocab.get('▁대선'))
print([tokenizer.encode(token) for token in result])

['▁', '너는', '▁', '내년', '▁', '대선', '▁', '때', '▁', '투표', 'ᄒ', 'ᅡᆯ', '▁', '수', '▁', 'ᄋ', 'ᅵ', 'ᆻ', 'ᄋ', 'ᅥ', '?']
1654
[[517, 0, 0], [517, 0, 0, 0], [517, 0, 0], [517, 0, 0, 0], [517, 0, 0], [517, 0, 0, 0], [517, 0, 0], [517, 0, 0, 0], [517, 0, 0], [517, 0, 0, 0], [517, 493, 0, 0], [517, 0, 0, 0], [517, 0, 0], [517, 0, 0, 0], [517, 0, 0], [517, 491, 0, 0], [517, 494, 0, 0], [517, 0, 0, 0], [517, 491, 0, 0], [517, 0, 0, 0], [633, 0, 0]]


In [ ]:
from sklearn.model_selection import train_test_split
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(model,  dr_rate=0.5).to(device)

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader

#pytorch model 저장을 위한 함수
def save_model(model, optimizer, epoch):
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': epoch,
    }, output_dir+f'telloment_senti_{epoch}.pth')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
train_history=[]
test_history=[]
loss_history=[]
model_dir = '/content/drive/MyDrive/dev/Telloment/sentiment-classification/result_dual_dataset/model/'
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_auto(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_auto(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    save_model(model, optimizer, e+1)
    torch.save(model, model_dir+f'telloment_senti_{epoch}.pth')
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


  0%|          | 0/1211 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.4203333854675293 train acc 0.328125


In [ ]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0: #분
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이") #슬
            elif np.argmax(logits) == 2:
                test_eval.append("분노가") #행
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이") #중립이

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [ ]:
sentence = '집에 가고싶다'
predict(sentence)
sentence = '엘리베이터에 갇혔었어'
predict(sentence)
sentence = '완전 럭키야!'
predict(sentence)
sentence = '모기는 왤케 많은거야'
predict(sentence)

In [ ]:
#pytorch model load을 위한 함수
def load_model(epoch:int):
  torch.load(output_dir+f'telloment_senti_{epoch}.pth')

In [ ]:
from google.colab import runtime
t